In [2]:
import json
import pandas as pd
import re
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

from keras.preprocessing import sequence
from keras.utils import np_utils

from tensorflow.keras.models import load_model

import urllib3
import ast

In [ ]:
# json파일 불러옴 (85개 파일)
len_rawdata=0
raw_data={"utterance":[]}

for k in range(0,100):
    if k <100:
        name="DKSR200000"+str(k)+".json"
    elif 100<= k and k<1000:
        name="DKSR20000"+str(k)+".json"
    try:
        f = open("[라벨]경상도_학습데이터_1/"+name, encoding="UTF-8")
        data = json.loads(f.read())
        raw_data["utterance"].extend(data["utterance"])
        #print(data["speaker"][1]["age"],data["speaker"][1]["birthplace"],data["speaker"][1]["principal_residence"])
        len_rawdata += 1

    except:
        #print("실패")
        pass    

In [7]:
# 사투리, 표준어 사전 생성 함수
def make_dict(raw_data):

    dialect_word = []
    standard_word = [] 
    all_std_word=[]

    filter = re.compile('[^가-힣+]')    
    
    for i in range(len(raw_data["utterance"])):
        #print(raw_data["utterance"])
        eojeol = raw_data["utterance"][i]["eojeolList"]
            
        for k in range(len(eojeol)):
            if eojeol[k]["isDialect"] == True:
                # print(eojeol[k]["eojeol"])
                # print(dialect_word)
                if filter.sub('', eojeol[k]['eojeol']) not in dialect_word :
                    dlt = filter.sub('', eojeol[k]['eojeol'])
                    std = filter.sub('', eojeol[k]['standard'])
                    if dlt=="" or std=="":
                        #print("dd", eojeol[k]['eojeol'], "dd")
                        pass
                    else:
                        dialect_word.append(dlt)
                        standard_word.append(std)
            else:
                if eojeol[k]["standard"] not in standard_word:
                    word = filter.sub('', eojeol[k]['standard'])
                    if word == "":
                        pass
                        #print("dd", eojeol[k]['eojeol'], "dd")
                    else:
                        all_std_word.append(word)

    word_dictionary = pd.DataFrame([dialect_word, standard_word])
    word_dictionary = word_dictionary.transpose()
    word_dictionary.columns = ['dialect', 'standard']
    dialect_dictionary = word_dictionary['dialect']
    standard_dictionary = word_dictionary['standard']

    index_to_dialect = dialect_dictionary.to_dict()
    index_to_standard = standard_dictionary.to_dict() #{인덱스:"단어"}

    dialect_to_index = {v:k for k,v in index_to_dialect.items()}
    standard_to_index = {v:k for k,v in index_to_standard.items()}#{"단어":인덱스}
    
    return dialect_to_index,standard_to_index,index_to_dialect,index_to_standard,all_std_word   

In [178]:
dialect_to_index,standard_to_index,index_to_dialect,index_to_standard,all_std_word = make_dict(raw_data)

389
349
389
389


In [9]:
# 사투리 표준어 쌍 생성 함수
def make_pair(raw_data):
    filter = re.compile('[^가-힣+]')

    dialect_form = []
    standard_form = []

    for i in range(len(raw_data["utterance"])):
        sentence = raw_data["utterance"][i]
        
        for j in range(len(sentence["eojeolList"])):
            #print(sentence["eojeolList"][0]['eojeol'])
            if sentence["eojeolList"][j]['isDialect'] == True:
                try : 
                    dialect = sentence["eojeolList"][j]["eojeol"]
                    standard = sentence["eojeolList"][j]["standard"]
                    
                    dialect = filter.sub('', dialect)
                    standard = filter.sub('', standard)
                    
                    dialect_idx = dialect_to_index[dialect]
                    #standard_idx = standard_to_index[standard]
                    standard_idx = [k for k, v in index_to_standard.items() if v == standard][0]

                    dialect_form.append(dialect_idx)
                    standard_form.append(standard_idx)
                except:
                    pass
            
    result = pd.DataFrame(data = {"dialect_words":dialect_form, "standard_words": standard_form})

    return result               

In [10]:
pairs = make_pair(raw_data)
pairs[:10]

,dialect_words,standard_words
0,0,0
1,1,1
2,2,0
3,0,0
4,3,3
5,1,1
6,0,0
7,0,0
8,4,4
9,0,0


In [11]:
# 사투리 사전에 존재하는지 확인하는 함수
def exist_in_dialect(x):

    if x in dialect_to_index:
        return True
    else:
        return False

In [12]:
trans_xtrain = np_utils.to_categorical(pairs["dialect_words"])
trans_ytrain = np_utils.to_categorical(pairs["standard_words"])
len(trans_xtrain[0])

389

In [13]:
translate_model = Sequential()
translate_model.add(Dense(256, input_dim = 389, activation="relu"))
translate_model.add(Dense(389, activation="softmax"))

In [14]:
translate_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
translate_model.fit(trans_xtrain, trans_ytrain, batch_size=100, epochs=50)

Epoch 1/50
12/12 [==============================] - 1s 3ms/step - loss: 5.9201 - accuracy: 0.2145
Epoch 2/50
12/12 [==============================] - 0s 4ms/step - loss: 5.7463 - accuracy: 0.5776
Epoch 3/50
12/12 [==============================] - 0s 8ms/step - loss: 5.5435 - accuracy: 0.5835
Epoch 4/50
12/12 [==============================] - 0s 7ms/step - loss: 5.2657 - accuracy: 0.5701
Epoch 5/50
12/12 [==============================] - 0s 14ms/step - loss: 4.8784 - accuracy: 0.5384
Epoch 6/50
12/12 [==============================] - 0s 5ms/step - loss: 4.3624 - accuracy: 0.5217
Epoch 7/50
12/12 [==============================] - 0s 6ms/step - loss: 3.7244 - accuracy: 0.4900
Epoch 8/50
12/12 [==============================] - 0s 5ms/step - loss: 3.0981 - accuracy: 0.4841
Epoch 9/50
12/12 [==============================] - 0s 5ms/step - loss: 2.6072 - accuracy: 0.5008
Epoch 10/50
12/12 [==============================] - 0s 5ms/step - loss: 2.2915 - accuracy: 0.5609
Epoch 11/50
12/12 

In [16]:
# 사투리 단어 번역 함수
def translate(word):
    index = dialect_to_index[word]
    one_hot = np_utils.to_categorical(index,len(trans_xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
    #print(translate_model.predict(np.array([one_hot])))
    pred_index = np.argmax(translate_model.predict(np.array([one_hot])))
    print(pred_index)
    #print(len(translate_model.predict(np.array([one_hot]))[0]))
    res = index_to_standard[pred_index]

    return res

In [61]:
translate("인자")

0


'이제'

In [17]:
# 품사 사전
a_tag = ['NP','VP','AP','VNP','DP','IP','X','L','R']
b_tag = ['SBJ','OBJ','MOD','AJT','CMP','CNJ']
c_tag = []
pos_dict = {}
for i in a_tag:
    for j in b_tag:
        c_tag.append(i+"_"+j)
c_tag.extend(a_tag)
c_tag.extend(b_tag)
for i in range(len(c_tag)):
    pos_dict[c_tag[i]] = i

In [18]:
# 데이터의 전체 어절을 딕셔너리로 만듦
def make_eojeol(index_to_standard, dialect_to_index, all_std_word):

    eojeol_list=[]
    st_list = []
    di_list = []
    
    for k in index_to_standard.values():
        st_list.append(k)
    print(len(st_list))
 

    for v in dialect_to_index:
        di_list.append(v)
    print(len(di_list))    

    for i, j in zip(st_list, di_list):
        eojeol_list.append(i)
        eojeol_list.append(j + "0")
        
    for i in all_std_word:
        eojeol_list.append(i) 

    eojeol_df = pd.DataFrame(eojeol_list, columns=None)
    eojeol_dict = eojeol_df.to_dict()
    eojeol_dict = eojeol_dict[0]
    eojeol_dict = {v:k for k,v in eojeol_dict.items()}

    #print(eojeol_dict)

    return eojeol_dict

In [19]:
eojeol_dict = make_eojeol(index_to_standard, dialect_to_index, all_std_word)

389
389


In [64]:
len(eojeol_dict)

31682

In [20]:
# 동의어 사전 만드는 함수
def make_synonym_list(eojeol_dict):
    synonym_list=[]
    for i in eojeol_dict.keys():
        if i[-1]=="0":
            for j in eojeol_dict.keys():
                tmp=i[:-1]
                if tmp==j:
                    synonym_list.append(tmp)
                    #print(i,j)
        else:
            pass
    return synonym_list

In [21]:
synonym_list = make_synonym_list(eojeol_dict)

In [65]:
synonym_list[:10]

['그래', '그니까', '땜에', '있나', '먹었다', '아이가', '내', '될려면', '따른', '그러고']

In [22]:
# 단어와 연관된 단어의 품사를 API를 사용해서 가져오기
def get_mods_pos(std_str, dlt_str):
    
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"

    accessKey = "aba3a45d-3318-4061-8a5d-799c8521b082"
    analysisCode = "dparse"

    text = std_str
    
    requestJson = {
        "access_key": accessKey,
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }
          
    http = urllib3.PoolManager()

    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )

    tmp = ast.literal_eval(response.data.decode('utf-8'))

    words = []
    poses = []
    
    #try : 
    if tmp["result"] == -1:

        print(tmp)

    std_list = std_str.split(". ")
    dlt_list = dlt_str.split(". ")
    key = True
    ket = False
    for t in range(len(std_list)-1):
        tmp["return_object"]["sentence"][t]

        std = std_list[t].split()
        dlt = dlt_list[t].split()

        try : 
            for i in range(len(tmp["return_object"]["sentence"][t]["dependency"])):
                dict_with_mods = tmp["return_object"]["sentence"][t]["dependency"][i]["mod"]
                # print("-------")
                # print(tmp["return_object"]["sentence"][t]["dependency"])
                #print("현재단어",tmp["return_object"]["sentence"][t]["dependency"][i]["text"])
                
                for m in dict_with_mods:
                    #print("모드있음")
                    #print(i)
                    if tmp["return_object"]["sentence"][t]["dependency"][i]["text"] != ".":
                        if std[i] != dlt[i]:
                            #print("사투리임")
                            # print(words)
                            
                            if dlt[i]+"0" in eojeol_dict.keys():
                                #print("동의어 사투리라서 두개 다 넣음", dlt[i], std[i])                            
                        
                                words.append(eojeol_dict[tmp["return_object"]["sentence"][t]["dependency"][int(m)]["text"]])
                                key=True
                                poses.append(pos_dict[tmp["return_object"]["sentence"][t]["dependency"][i]["label"]])
                                key=False

                                words.append(eojeol_dict[dlt[i]+"0"])
                                key=True
                                poses.append(pos_dict[tmp["return_object"]["sentence"][t]["dependency"][int(m)]["label"]])
                                key=False

                                words.append(eojeol_dict[std[i]])
                                key=True
                                poses.append(pos_dict[tmp["return_object"]["sentence"][t]["dependency"][int(m)]["label"]])
                                key=False
                            # else:
                            #     # print("사투리 사전에 없음")

                        else:
                            #print("사투리아님")
                            if std[i]+"0" in eojeol_dict.keys():
                                
                                words.append(eojeol_dict[std[i]])
                                key=True
                                poses.append(pos_dict[tmp["return_object"]["sentence"][t]["dependency"][int(m)]["label"]])
                                key=False

                                words.append(eojeol_dict[tmp["return_object"]["sentence"][t]["dependency"][int(m)]["text"]])
                                key=True
                                poses.append(pos_dict[tmp["return_object"]["sentence"][t]["dependency"][i]["label"]])
                                key=False
            
            #print("성공")

        except:
            #print("실패")
            if key == True:
                words.pop()
            #print(len(words), len(poses))

    # except:
    #     print(tmp)

    #print(words, poses)
    return words, poses

In [66]:
len(raw_data["utterance"])

22500

In [23]:
# 단어별 인덱스와 연결된 단어의 품사 인덱스 

def eojeol_dependence(raw_data, synonym_list):
    words=[]
    poses=[]
    std_str=""
    dlt_str=""
    count = 0
    num=0
    for i in raw_data["utterance"]:

        # 문장
        std = i["standard_form"]
        dlt = i["dialect_form"]
        
        filter = re.compile('[^가-힣 +]')

        # 불용어 제거 문장
        dlt = filter.sub('', dlt)
        std = filter.sub('', std)

        # 문장 이어붙임
        total = dlt.split() + std.split()

        # 이어붙인 문장의 어절을 돌면서
        for eojeol in total:
            if eojeol in synonym_list: # 동의어 목록에 있는 어절을 발견하면
                # get_mods_pos로 넘길 str에 추가
                dlt_str = dlt_str + dlt + ". "
                std_str = std_str + std + ". "
                count = count + 1
                break

        if count >= 200: #문장 총 개수 22500개, 동의어가 있는 문장이 200개가 넘으면 API 사용 -> API 사용횟수 : 약 100회   
            #print(num) 
            num +=1
            word, pos = get_mods_pos(std_str, dlt_str)
            words.extend(word)
            poses.extend(pos)
            count = 0
            std_str = ""
            dlt_str = ""

    df = pd.DataFrame(data = {"pos":poses, "word":words })
        
    return df

In [73]:
train_df[train_df["word"] == 148241]

,pos,word
923,54,148241
1868,54,148241
1870,57,148241
1872,8,148241
1874,54,148241
1878,8,148241
2067,54,148241
4785,58,148241
5176,8,148241
5700,54,148241


In [487]:
eojeol_dict["아이가"]

148241

In [ ]:
train_df = eojeol_dependence(raw_data, synonym_list)

In [71]:
train_df[["pos", "word"]]

,pos,word
0,55,155171
1,55,953
2,55,149387
3,54,116685
4,56,149387
...,...,...
6128,55,156279
6129,3,155743
6130,8,156026
6131,55,156016


In [493]:
train_df.to_csv("train_df.csv")

In [72]:
train_df = pd.read_csv("train_df.csv")
train_df = train_df[["pos", "word"]]

In [25]:
len(set(train_df["word"]))

1573

In [26]:
# 원핫인코딩을 하기 위해 
index_word={} # 
num=0
for i in train_df["word"]:
    if i not in index_word.keys():
        index_word[i] = num
        num += 1
#print(index_word)
word_index_col=[]
for i in train_df["word"]:
    word_index_col.append(index_word[i])


In [74]:
train_df["word_index_col"] = word_index_col


,pos,word,word_index_col
0,55,155171,0
1,55,953,1
2,55,149387,2
3,54,116685,3
4,56,149387,2
...,...,...,...
6128,55,156279,52
6129,3,155743,8
6130,8,156026,1572
6131,55,156016,25


In [75]:
train_df[:30]

,pos,word,word_index_col
0,55,155171,0
1,55,953,1
2,55,149387,2
3,54,116685,3
4,56,149387,2
5,54,141550,4
6,56,149387,2
7,54,104230,5
8,59,144143,6
9,57,151458,7


In [545]:
# 단어별 인덱스와 연결된 단어의 품사 인덱스
#def eojeol_dependence_():

    # 언어 분석 기술(문어)
    # openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    # accessKey = "aba3a45d-3318-4061-8a5d-799c8521b082"
    # analysisCode = "dparse"

    # text = "엑소브레인은 내 몸 바깥에 있는 인공 두뇌라는 뜻으로, 세계 최고인공지능 기술 선도라는 비전을 달성하기 위한 과학기술정보통신부 소프트웨어 분야의 국가 혁신기술 개발형 연구개발 과제이다."
    # requestJson = {
    #     "access_key": accessKey,
    #     "argument": {
    #         "text": text,
    #         "analysis_code": analysisCode
    #     }
    # }
    # http = urllib3.PoolManager()
    # response = http.request(
    #     "POST",
    #     openApiURL,
    #     headers={"Content-Type": "application/json; charset=UTF-8"},
    #     body=json.dumps(requestJson)
    # )

    # s=ast.literal_eval(response.data.decode('utf-8'))

    # dep=s['return_object']['sentence'][0]['dependency']
    
    # eojeol_d=[]
    # pos_d=[]

    # re_eojeol_dict = {v:k for k,v in eojeol_dict.items()} #{인덱스:문자}
        
    # for i in range(len(dep)): #dep(딕셔너리)길이만큼 반복 23
    #     if dep[i]['text'] in eojeol_dict:
    #         eojeol_d.append(eojeol_dict.values())
    #     for j in range(len(dep[i]['mod'])):
    #         mod=dep[i]['mod']
    #         print(mod[j])
    #         if len(mod[j]) > 0:
    #             pos=re_eojeol_dict.get(mod[j])#결과값:문자
    #             #mod가 가리키는 어절의 품사를 확인하는 방법..
    #             if dep[i]['label'] in pos_dict:
    #                 pos_d.append(pos_dict.values())
    #             else:
    #                 return None
                
    # result = pd.DataFrame(data = {"word_dic":eojeol_d, "mod_dic":pos_d})
    # return result



In [28]:
synonym_xtrain = np_utils.to_categorical(train_df["pos"])
synonym_ytrain = np_utils.to_categorical(train_df["word_index_col"])
len(synonym_ytrain[0])

1573

In [29]:
print(synonym_xtrain.shape)
print(synonym_ytrain.shape)

(6133, 60)
(6133, 1573)


In [504]:
synonym_model = Sequential()
synonym_model.add(Dense(512, input_dim = 60, activation="relu"))
synonym_model.add(Dense(256,activation="relu"))
synonym_model.add(Dense(128,activation="relu"))
synonym_model.add(Dense(1573, activation="softmax"))

In [505]:
synonym_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [506]:
synonym_model.fit(synonym_xtrain, synonym_ytrain, batch_size=500, epochs=50)

Epoch 1/50
13/13 [==============================] - 1s 26ms/step - loss: 7.2769 - accuracy: 0.1159
Epoch 2/50
13/13 [==============================] - 0s 31ms/step - loss: 6.3156 - accuracy: 0.1500
Epoch 3/50
13/13 [==============================] - 0s 31ms/step - loss: 5.5938 - accuracy: 0.1314
Epoch 4/50
13/13 [==============================] - 0s 30ms/step - loss: 5.4219 - accuracy: 0.1660
Epoch 5/50
13/13 [==============================] - 0s 31ms/step - loss: 5.2840 - accuracy: 0.1798
Epoch 6/50
13/13 [==============================] - 0s 29ms/step - loss: 5.1802 - accuracy: 0.1794
Epoch 7/50
13/13 [==============================] - 0s 31ms/step - loss: 5.0957 - accuracy: 0.1875
Epoch 8/50
13/13 [==============================] - 0s 28ms/step - loss: 5.0202 - accuracy: 0.1935
Epoch 9/50
13/13 [==============================] - 0s 31ms/step - loss: 4.9528 - accuracy: 0.1931
Epoch 10/50
13/13 [==============================] - 0s 28ms/step - loss: 4.8964 - accuracy: 0.1944
Epoch 11/

In [77]:
eojeol_dict["아이가"]

148241

In [78]:
eojeol_dict["아이가0"]

37

In [537]:
synonym_model.save("synonym_model.h5")

In [30]:
from tensorflow.keras.models import load_model
synonym_model = load_model("synonym_model.h5")

In [34]:
synonym_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               31232     
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_11 (Dense)             (None, 1573)              202917    
Total params: 398,373
Trainable params: 398,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 단어와 연관된 단어의 품사 추출
def get_mods(sentence, num):
    #print(sentence, num)
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"

    accessKey = "aba3a45d-3318-4061-8a5d-799c8521b082"
    
    #bd6d2bff-4e44-4035-8591-947fda262051
    analysisCode = "dparse"

    text = sentence
    
    requestJson = {
        "access_key": accessKey,
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }

    http = urllib3.PoolManager()

    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )

    tmp = ast.literal_eval(response.data.decode('utf-8'))
    #print(tmp)
    dict_with_mods = tmp["return_object"]["sentence"][0]["dependency"]
    id_dict={}
    print(dict_with_mods)
    for i in dict_with_mods[num]["mod"]:
        id_dict[dict_with_mods[int(i)]["weight"]] = int(i)
        #print(i)
    id = id_dict[max(id_dict)]
    
    pos = dict_with_mods[id]["label"]

    
    return pos, dict_with_mods[id]["text"]

In [137]:
# 동의어 처리를 포함한 사투리 번역 함수
def predict_standard(sentence):
    global index_to_standard
    res = ""
    mark = ""
    if sentence[-1] in [".", "?", "!"]:
        mark = sentence[-1]
        sentence = sentence[:-1]

    sentence_sep = sentence.split()
    
        
    #print(sentence_sep)
    for i in range(len(sentence_sep)):
        #print(sentence_sep[i])
        if exist_in_dialect(sentence_sep[i]) == True: #사투리 사전에 있으면
            if sentence_sep[i] in eojeol_dict.keys(): #해당 단어가 전체 사전에 있으면
                #print(sentence_sep[i] )
                pos, _ = get_mods(sentence, i) #연관된 단어의 품사 받아옴
                pos_idx = pos_dict[pos] #품사의 인덱스
                
                word = sentence_sep[i]
                print(i, word) 
                print(eojeol_dict[word+"0"])
                print(eojeol_dict[word])
                word0_idx = index_word[ eojeol_dict[word+"0"]] #단어0의 인덱스
                word_idx = index_word[eojeol_dict[word]] #단어의 인덱스
                #print(word0_idx)
                # temp = result[word_idx]
                # print(type(temp))

                one_hot = np_utils.to_categorical(pos_idx,len(synonym_xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
                #print((one_hot))
                
                percentage = synonym_model.predict(np.array([one_hot])) # 동의어 처리 모델
                #print("ㅇㅇ",percentage[0])

                
                res_idx = max(percentage[0][word0_idx], percentage[0][word_idx]) # 둘 중 더 큰 확률값
                print("사투리확률", percentage[0][word0_idx])
                print("표준어확률", percentage[0][word_idx])
                
                if percentage[0][word0_idx] > percentage[0][word_idx]:

                    word = translate(word)
                else: 
                    pass

                res = res + " " + word
                #res = res + " " + index_to_standard[pred_index]
            else: #해당 단어가 전체 사전에 없으면 -> 동의어가 없으므로 바로 번역 모델에 넣음
                
                word = sentence_sep[i]
                res = res + " " + translate(word)
        else:
            word = sentence_sep[i]
            #print(word,"사투리가아님")
            res = res + " " + word
    if mark != "":
        res = res + mark
            
    return res

In [166]:
train_df[train_df["pos"]==55] #pos "VP"

,pos,word,word_index_col
0,55,155171,0
1,55,953,1
2,55,149387,2
16,55,155743,8
22,55,155743,8
...,...,...,...
6120,55,155495,13
6122,55,151880,1571
6124,55,155902,219
6128,55,156279,52


In [159]:
train_df[train_df["word"]==37] # 사투리 "아이가"

,pos,word,word_index_col
274,55,37,113
836,4,37,113
856,55,37,113
876,55,37,113
931,55,37,113
947,55,37,113
972,55,37,113
1219,55,37,113
1237,55,37,113
1257,55,37,113


In [158]:
train_df[train_df["word"]==148241] #표준어 "아이가"

,pos,word,word_index_col
923,54,148241,330
1868,54,148241,330
1870,57,148241,330
1872,8,148241,330
1874,54,148241,330
1878,8,148241,330
2067,54,148241,330
4785,58,148241,330
5176,8,148241,330
5700,54,148241,330


In [508]:
translate("있다")

85


'있지'

In [80]:
eojeol_dict["아이가"]

148241

In [517]:
eojeol_dict["있다0"]

171

In [514]:
train_df[train_df["word"]==171] #사투리 있다

,pos,word,word_index_col
1330,55,171,458
1333,56,171,458
1336,0,171,458
4951,55,171,458
4954,59,171,458
4957,1,171,458


In [179]:
train_df[train_df["word"]==155786] #표준어 있다

,pos,word,word_index_col
35,0,155786,23
56,0,155786,23
76,0,155786,23
206,0,155786,23
323,3,155786,23
...,...,...,...
5955,56,155786,23
5957,56,155786,23
5959,3,155786,23
6007,0,155786,23


In [32]:
predict_standard("뭐라카노")

' 뭐라카노'

In [96]:
predict_standard("오늘 수업을 하나")

[{'id': 0.0, 'text': '오늘', 'head': 2.0, 'label': 'NP', 'mod': [], 'weight': 0.0825357}, {'id': 1.0, 'text': '수업을', 'head': 2.0, 'label': 'NP_OBJ', 'mod': [], 'weight': 0.667639}, {'id': 2.0, 'text': '하나', 'head': -1.0, 'label': 'NP', 'mod': [0.0, 1.0], 'weight': 0.0266307}]
2 하나
203
155170
사투리확률 0.0033866374
표준어확률 0.0009904605
101


' 오늘 수업을 하냐'

In [ ]:
predict_standard("내가 얼른 글로 갈게")

In [102]:
predict_standard("많이 뭇다 아이가")

[{'id': 0.0, 'text': '많이', 'head': 1.0, 'label': 'AP', 'mod': [], 'weight': 0.789286}, {'id': 1.0, 'text': '뭇다', 'head': 2.0, 'label': 'VP', 'mod': [0.0], 'weight': 0.15832}, {'id': 2.0, 'text': '아이가', 'head': -1.0, 'label': 'NP_SBJ', 'mod': [1.0], 'weight': 0.054921}]
2 아이가
37
148241
사투리확률 0.005546723
표준어확률 1.0859945e-05
41


' 많이 뭇다 않아'

In [152]:
predict_standard("맞제 이거 약간 모의 투자 느낌.")

43


' 맞지 이거 약간 모의 투자 느낌.'

In [150]:
predict_standard("내가 집에서 손으로 다 해 먹거든예")

' 내가 집에서 손으로 다 해 먹거든예'

In [139]:
predict_standard("최고의 리더는 글을 쓴다.") #표준어 쓴다

[{'id': 0.0, 'text': '최고의', 'head': 1.0, 'label': 'NP_MOD', 'mod': [], 'weight': 0.58665}, {'id': 1.0, 'text': '리더는', 'head': 3.0, 'label': 'NP_SBJ', 'mod': [0.0], 'weight': 0.747763}, {'id': 2.0, 'text': '글을', 'head': 3.0, 'label': 'NP_OBJ', 'mod': [], 'weight': 0.732951}, {'id': 3.0, 'text': '쓴다', 'head': -1.0, 'label': 'VP', 'mod': [1.0, 2.0], 'weight': 0.23426}]
3 쓴다
89
88112
사투리확률 2.4423378e-05
표준어확률 0.0022023644


' 최고의 리더는 글을 쓴다.'

In [138]:
predict_standard("최고의 리더니까 최고의 글을 쓴다 아이가?") #사투리 쓴다, 사투리 아이가

[{'id': 0.0, 'text': '최고의', 'head': 1.0, 'label': 'NP_MOD', 'mod': [], 'weight': 0.608439}, {'id': 1.0, 'text': '리더니까', 'head': 4.0, 'label': 'VNP', 'mod': [0.0], 'weight': 0.37775}, {'id': 2.0, 'text': '최고의', 'head': 3.0, 'label': 'NP_MOD', 'mod': [], 'weight': 0.56849}, {'id': 3.0, 'text': '글을', 'head': 4.0, 'label': 'NP_OBJ', 'mod': [2.0], 'weight': 0.688654}, {'id': 4.0, 'text': '쓴다', 'head': 5.0, 'label': 'VP', 'mod': [1.0, 3.0], 'weight': 0.336526}, {'id': 5.0, 'text': '아이가', 'head': -1.0, 'label': 'NP_SBJ', 'mod': [4.0], 'weight': 0.0141848}]
4 쓴다
89
88112
사투리확률 0.0025564772
표준어확률 4.2879005e-06
44
[{'id': 0.0, 'text': '최고의', 'head': 1.0, 'label': 'NP_MOD', 'mod': [], 'weight': 0.608439}, {'id': 1.0, 'text': '리더니까', 'head': 4.0, 'label': 'VNP', 'mod': [0.0], 'weight': 0.37775}, {'id': 2.0, 'text': '최고의', 'head': 3.0, 'label': 'NP_MOD', 'mod': [], 'weight': 0.56849}, {'id': 3.0, 'text': '글을', 'head': 4.0, 'label': 'NP_OBJ', 'mod': [2.0], 'weight': 0.688654}, {'id': 4.0, 'text': '쓴

' 최고의 리더니까 최고의 글을 쓰지 않아?'

In [153]:
predict_standard("게임을 그렇게 대충 해가지고 이기겠나?")

[{'id': 0.0, 'text': '게임을', 'head': 3.0, 'label': 'NP_OBJ', 'mod': [], 'weight': 0.736509}, {'id': 1.0, 'text': '그렇게', 'head': 3.0, 'label': 'VP_AJT', 'mod': [], 'weight': 0.575393}, {'id': 2.0, 'text': '대충', 'head': 3.0, 'label': 'AP', 'mod': [], 'weight': 0.579023}, {'id': 3.0, 'text': '해가지고', 'head': 4.0, 'label': 'VP', 'mod': [0.0, 1.0, 2.0], 'weight': 0.532814}, {'id': 4.0, 'text': '이기겠나', 'head': -1.0, 'label': 'VP', 'mod': [3.0], 'weight': 0.10447}]
3 해가지고
643
156118
사투리확률 0.003530945
표준어확률 0.1711169
95


' 게임을 그렇게 대충 해가지고 이길까?'

In [154]:
predict_standard("게임을 그렇게 대충 해가지고 이기긌나?")

[{'id': 0.0, 'text': '게임을', 'head': 3.0, 'label': 'NP_OBJ', 'mod': [], 'weight': 0.736048}, {'id': 1.0, 'text': '그렇게', 'head': 3.0, 'label': 'VP_AJT', 'mod': [], 'weight': 0.580144}, {'id': 2.0, 'text': '대충', 'head': 3.0, 'label': 'AP', 'mod': [], 'weight': 0.578524}, {'id': 3.0, 'text': '해가지고', 'head': 4.0, 'label': 'VP', 'mod': [0.0, 1.0, 2.0], 'weight': 0.516002}, {'id': 4.0, 'text': '이기긌나', 'head': -1.0, 'label': 'VP', 'mod': [3.0], 'weight': 0.100202}]
3 해가지고
643
156118
사투리확률 0.003530945
표준어확률 0.1711169


' 게임을 그렇게 대충 해가지고 이기긌나?'

In [110]:
dialect_to_index["아이가"]

18

In [112]:
index_to_standard[18]

'않니'

In [117]:
eojeol_dict["갖고"]

147024

In [ ]:
index_word[154389]

In [111]:
pairs[pairs["dialect_words"] == 18]

,dialect_words,standard_words
37,18,18
137,18,41
141,18,41
145,18,41
147,18,41
150,18,41
177,18,41
182,18,41
185,18,41
197,18,41


In [157]:
predict_synonym("어릴때 옆집 살던 가는 뭐하노?")

[{'id': 0.0, 'text': '어릴때', 'head': 4.0, 'label': 'VP', 'mod': [], 'weight': 0.238896}, {'id': 1.0, 'text': '옆집', 'head': 2.0, 'label': 'NP_SBJ', 'mod': [], 'weight': 0.0295369}, {'id': 2.0, 'text': '살던', 'head': 3.0, 'label': 'VP_MOD', 'mod': [1.0], 'weight': 0.42003}, {'id': 3.0, 'text': '가는', 'head': 4.0, 'label': 'VP_MOD', 'mod': [2.0], 'weight': 0.572229}, {'id': 4.0, 'text': '뭐하노', 'head': -1.0, 'label': 'VNP', 'mod': [0.0, 3.0], 'weight': 0.00104964}]
3 가는
281
155743
사투리확률 0.002120091
표준어확률 0.004655487


' 어릴때 옆집 살던 가는 뭐하노?'

In [142]:
predict_standard("착한 아이가 저기 앉아 있다 아이가?")

[{'id': 0.0, 'text': '착한', 'head': 1.0, 'label': 'VP_MOD', 'mod': [], 'weight': 0.625791}, {'id': 1.0, 'text': '아이가', 'head': 3.0, 'label': 'NP_SBJ', 'mod': [0.0], 'weight': 0.695618}, {'id': 2.0, 'text': '저기', 'head': 3.0, 'label': 'NP_AJT', 'mod': [], 'weight': 0.573246}, {'id': 3.0, 'text': '앉아', 'head': 4.0, 'label': 'VP', 'mod': [1.0, 2.0], 'weight': 0.750703}, {'id': 4.0, 'text': '있다', 'head': 5.0, 'label': 'VP', 'mod': [3.0], 'weight': 0.271241}, {'id': 5.0, 'text': '아이가', 'head': -1.0, 'label': 'NP_SBJ', 'mod': [4.0], 'weight': 0.0247723}]
1 아이가
37
148241
사투리확률 8.952507e-09
표준어확률 0.0059404573
[{'id': 0.0, 'text': '착한', 'head': 1.0, 'label': 'VP_MOD', 'mod': [], 'weight': 0.625791}, {'id': 1.0, 'text': '아이가', 'head': 3.0, 'label': 'NP_SBJ', 'mod': [0.0], 'weight': 0.695618}, {'id': 2.0, 'text': '저기', 'head': 3.0, 'label': 'NP_AJT', 'mod': [], 'weight': 0.573246}, {'id': 3.0, 'text': '앉아', 'head': 4.0, 'label': 'VP', 'mod': [1.0, 2.0], 'weight': 0.750703}, {'id': 4.0, 'text': '있다

' 착한 아이가 저기 앉아 있다 않아?'

In [141]:
predict_standard("너처럼 어린 아이가 무슨 운전 면허증을 따니?")

[{'id': 0.0, 'text': '너처럼', 'head': 6.0, 'label': 'NP_AJT', 'mod': [], 'weight': 0.711283}, {'id': 1.0, 'text': '어린', 'head': 2.0, 'label': 'VP_MOD', 'mod': [], 'weight': 0.607093}, {'id': 2.0, 'text': '아이가', 'head': 6.0, 'label': 'NP_SBJ', 'mod': [1.0], 'weight': 0.841032}, {'id': 3.0, 'text': '무슨', 'head': 5.0, 'label': 'DP', 'mod': [], 'weight': 0.38138}, {'id': 4.0, 'text': '운전', 'head': 5.0, 'label': 'NP', 'mod': [], 'weight': 0.327299}, {'id': 5.0, 'text': '면허증을', 'head': 6.0, 'label': 'NP_OBJ', 'mod': [3.0, 4.0], 'weight': 0.687231}, {'id': 6.0, 'text': '따니', 'head': -1.0, 'label': 'VP', 'mod': [0.0, 2.0, 5.0], 'weight': 0.0258534}]
2 아이가
37
148241
사투리확률 8.952507e-09
표준어확률 0.0059404573


' 너처럼 어린 아이가 무슨 운전 면허증을 따니?'